# Kernel Support Vector Machines

In this exercise sheet, we will implement a kernel SVM. Our implementation will be based on a generic quadratic programming optimizer provided in CVXOPT (`python-cvxopt` package, or directly from the website `www.cvxopt.org`). The SVM will then be tested on the UCI breast cancer dataset, a simple binary classification dataset accessible via the `scikit-learn` library.

## 1. Building the Gaussian Kernel (5 P)

As a starting point, we would like to implement the Gaussian kernel, which we will make use of in our kernel SVM implementation. It is defined as:
$$
k(x,x') = \exp \Big( -\frac{\|x-x'\|^2}{2 \sigma^2} \Big)
$$

* **Implement a function `getGaussianKernel` that returns for a Gaussian kernel of scale $\sigma$, the Gram matrix of the two data sets given as argument.**

In [1]:
import numpy,scipy,scipy.spatial

def getGaussianKernel(X1,X2,scale):
    ### TODO: REPLACE BY YOUR OWN CODE
    #import solutions
    #K = solutions.getGaussianKernel(X1,X2,scale)
    #return K
    
    #print("x:",X1.shape)
    #x: (284, 30)

    
    #K = numpy.exp(- (numpy.linalg.norm(X1 - X2) ** 2) / (2 * scale ** 2))
    
    K = numpy.zeros((X1.shape[0], X2.shape[0]))
    for i in range(X1.shape[0]):
        for j in range(X2.shape[0]):
            K[i, j] = numpy.exp(- (numpy.linalg.norm(X1[i,:] - X2[j,:]) ** 2) / (2 * (scale ** 2)))
            
            
    return K
    ###

[[1 2]
 [3 4]
 [5 6]]


(1, 2)

## 2. Building the Matrices for the CVXOPT Quadratic Solver (20 P)

We would like to learn a nonlinear SVM by optimizing its dual. An advantage of the dual SVM compared to the primal SVM is that it allows to use nonlinear kernels such as the Gaussian kernel. The dual SVM consists of solving the following quadratic program:

$$
\max_\alpha \sum_{i=1}^N \alpha_i - \frac12 \sum_{ij} \alpha_i \alpha_j y_i y_j k(x_i,x_j)
\qquad 
\text{subject to:}
\qquad 0 \leq \alpha_i \leq C \qquad \text{and} \qquad \sum_{i=1}^N \alpha_i y_i = 0.
$$

We would like to rely on a CVXOPT solver to obtain a solution to our SVM dual. The function `cvxopt.solvers.qp` solves an optimization problem of the type:

\begin{align*}
\min_{\boldsymbol{x}} \quad &\frac12 \boldsymbol{x}^\top P \boldsymbol{x} + \boldsymbol{q}^\top \boldsymbol{x}\\
\text{subject to} \quad & G \boldsymbol{x} \preceq \boldsymbol{h}\\
\text{and} \quad & A \boldsymbol{x} = \boldsymbol{b}.
\end{align*}

which is of similar form to our dual SVM (note that $\boldsymbol{x}$ will correspond to the parameters $(\alpha_i)_i$ of the SVM). We need to build the data structures (vectors and matrices) that makes solving this quadratic problem equivalent to solving our dual SVM.

* **Implement a function `getQPMatrices` that builds the matrices `P`, `q`, `G`, `h`, `A`, `b` (of type `cvxopt.matrix`) that need to be passed as argument to the optimizer `cvxopt.solvers.qp`.**

In [2]:
import cvxopt,cvxopt.solvers 
cvxopt.solvers.options['show_progress'] = False

def getQPMatrices(K,T,C):
    ### TODO: REPLACE BY YOUR CODE
    #import solutions
    #P,q,G,h,A,b = solutions.getQPMatrices(K,T,C)
    #return P,q,G,h,A,b
    
    # T is the label y
    N = len(T)
    #print("K,T,C,N:",K.shape,T.shape,C,N)
    #K,T,C,N: (284, 284) (284,) 10 284

    Y = T
    
    #P = T * T @ K
    #print(    numpy.outer(T, T).shape,K.shape)
    #print("Y[None],Y[:,None],K:",Y[None].shape,Y[:,None].shape,K.shape)
    #P = numpy.outer(Y, Y).dot(K)
    #print("T:",T.shape)
    #print("outer:",numpy.outer(Y, Y))
    #print("Y[:,None] @ Y[None]:",Y[:,None] @ Y[None])
    
    
    P =  (Y[:,None] @ Y[None]) * K
    q = -1 * numpy.ones((N,1))
    G = numpy.vstack((-1*numpy.eye(N),numpy.eye(N)))
    h = numpy.hstack((numpy.zeros(N),C * numpy.ones(N)))
    A = T[None,:]
    b = 0
    
    #print(P.shape,q.shape,G.shape,h.shape,A.shape,b)
    #(284, 284) (284, 1) (568, 284) (568,) (1, 284) 0.0


    
    P = cvxopt.matrix(P, tc='d')
    q = cvxopt.matrix(q, tc='d')
    G = cvxopt.matrix(G, tc='d')
    h = cvxopt.matrix(h, tc='d')
    A = cvxopt.matrix(A, tc='d')
    b = cvxopt.matrix(b, tc='d')


    
    return P,q,G,h,A,b





    ##ddds†#

      

## 3. Computing the Bias Parameters (10 P)

Given the parameters $(\alpha_i)_i$ the optimization procedure has found, the prediction of the SVM is given by:

$$
f(x) = \text{sign}\Big(\sum_{i=1}^N \alpha_i y_i k(x,x_i) + \theta\Big)
$$

Note that the parameter $\theta$ has not been computed yet. It can be obtained from any support vector that lies exactly on the margin, or equivalently, whose associated parameter $\alpha$ is not equal to $0$ or $C$. Calling one such vector "$x_M$", the parameter $\theta$ can be computed as:

$$
\theta =  y_M - \sum_{j=1}^N \alpha_j y_j k(x_M,x_j) 
$$

* **Implement a function `getTheta` that takes as input the Gram Matrix used for training, the label vector, the solution of our quadratic program, and the hyperparameter C. The function should return the parameter $\theta$.**

In [3]:
def getTheta(K,T,alpha,C):
    ### TODO: REPLACE BY YOUR CODE
    #import solutions
    #theta = solutions.getTheta(K,T,alpha,C)
    #print(K.shape,T.shape,alpha)
    #(284, 284) (284,) {'x': <284x1 matrix, tc='d'>, 'y': <1x1 matrix, tc='d'>, 's': <568x1 matrix, tc='d'>, 'z': <568x1 matrix, tc='d'>, 'status': 'optimal', 'gap': 7.118147899470116e-05, 'relative gap': 6.783961494705319e-07, 'primal objective': -104.92612472853241, 'dual objective': -104.9261959100114, 'primal infeasibility': 1.5101568542741277e-14, 'dual infeasibility': 5.925681618886442e-16, 'primal slack': 7.020236463770086e-08, 'dual slack': 6.403597588856221e-09, 'iterations': 9}

    
    alpha = numpy.array(alpha['x'])
    #print(alpha.shape) 
    #(284, 1)
    alpha = alpha.flatten()
    #print(alpha) 
    """
    [5.92118709e+00 2.56049320e-07 3.11798719e-07 3.16622871e-07
 1.08017991e+00 1.42118448e-07 7.80501476e-01 2.62696768e-07
    """

    
    idx = -1
    for i in range(len(alpha)):
        if 1e-6 < alpha[i] < C:
            idx = i
            break


    y_M = T[idx]
    theta = y_M - numpy.sum(alpha * T * K[idx,:])
    
    
    return theta
    ###

## 4. Implementing a class `GaussianSVM` (15 P)

All functions that are needed to learn the SVM have now been built. We would like to implement a SVM class that connects them and make the SVM easily usable. The class structure is given below and contains two functions, one for training the model, and one for applying it to test data.

* **Implement the function `fit` that makes use of the functions `getGaussianKernel`, `getQPMatrices`, `getTheta` you have already implemented. The function should learn the SVM model and store the support vectors, their label, $(\alpha_i)_i$ and $\theta$ into the object (`self`).**
* **Implement the function `predict` that makes use of the stored information to compute the SVM output for any new collection of data points**

In [6]:
class GaussianSVM:

    def __init__(self,C=1.0,scale=1.0):
        
        self.C , self.scale = C, scale
    
    def fit(self,X,T):

        ### TODO: REPLACE BY YOUR CODE
        #import solutions
        #solutions.fit(self,X,T)
        self.Y = T
        K = getGaussianKernel(X,X,self.scale)
        P,q,G,h,A,b = getQPMatrices(K,T,self.C)
        alpha = cvxopt.solvers.qp(P,q,G,h,A,b)
        self.alpha = numpy.array(alpha['x']).flatten()
        self.theta = getTheta(K,T,alpha,self.C)
        #print(self.alpha)
        
        sv_idx = numpy.logical_not(
            numpy.isclose(numpy.zeros(self.alpha.shape[0]), self.alpha, atol=1e-5)
        ) 
        '''
        
        for i in range(len(self.alpha)):
            if 1e-6 < self.alpha[i] < self.C:
                sv_idx.append(i)
                
        we dont need that because 0 < alpha < c has already in cvxopt 
        '''
        
                
        self.sv_idx = sv_idx 
        self.X = X[sv_idx] # it's sv
     
        ###
        
    def predict(self,X):
        
        ### TODO: REPLACE BY YOUR CODE
        #import solutions
        #Y = solutions.predict(self,X)
        K = getGaussianKernel(self.X,X,self.scale)
        #print(self.alpha.shape,self.Y.shape,K.shape) #(284,) (284,) (189, 284)
        #print(X.shape) #(284, 30)
        #print("self.sv_idx:",len(self.sv_idx)) #self.sv_idx: 189
        
       # Y = numpy.sign((self.alpha[i] * self.Y[i] * K[Y[i],:]) for i in self.sv_idx)
       # Y = numpy.sign(numpy.sum(self.alpha[i,None] * self.Y[i,None] * K[i, :] for i in self.sv_idx ) + self.theta)
        #print(self.alpha[1,None].shape, self.Y[i,None].shape , K[1, :].shape)
        alpha_sv = self.alpha[self.sv_idx]  
        Y_sv = self.Y[self.sv_idx]
        res = alpha_sv[:,None] * Y_sv[:,None] * K
        #print("res:",res.shape) #res: (189, 284)
        #print(numpy.sum(res,axis=0).shape)
        Y = numpy.sign(numpy.sum(res,axis=0) + self.theta)
        
       


        return Y
        ###

## 5. Analysis

The following code tests the SVM on some breast cancer binary classification dataset for a range of scale and soft-margin parameters. For each combination of parameters, we output the number of support vectors as well as the train and test accuracy averaged over a number of random train/test splits. Running the code below should take approximately 1-2 minutes.

In [7]:
import numpy,sklearn,sklearn.datasets,numpy

D = sklearn.datasets.load_breast_cancer()
X = D['data']
T = D['target']
T = (D['target']==1)*2.0-1.0

for scale in [30,100,300,1000,3000]:
    for C in [10,100,1000,10000]:
        
        acctrain,acctest,nbsvs = [],[],[]
        
        svm = GaussianSVM(C=C,scale=scale)
        
        for i in range(10):

            # Split the data
            R = numpy.random.mtrand.RandomState(i).permutation(len(X))
            Xtrain,Xtest = X[R[:len(R)//2]]*1,X[R[len(R)//2:]]*1
            Ttrain,Ttest = T[R[:len(R)//2]]*1,T[R[len(R)//2:]]*1

            # Train and test the SVM
            svm.fit(Xtrain,Ttrain)
            acctrain += [(svm.predict(Xtrain)==Ttrain).mean()]
            acctest  += [(svm.predict(Xtest)==Ttest).mean()]
            nbsvs += [len(svm.X)*1.0]

        print('scale=%9.1f  C=%9.1f  nSV: %4d  train: %.3f  test: %.3f'%(
            scale,C,numpy.mean(nbsvs),numpy.mean(acctrain),numpy.mean(acctest)))
    print('')

scale=     30.0  C=     10.0  nSV:  168  train: 0.997  test: 0.921
scale=     30.0  C=    100.0  nSV:  166  train: 1.000  test: 0.917
scale=     30.0  C=   1000.0  nSV:  166  train: 1.000  test: 0.917
scale=     30.0  C=  10000.0  nSV:  167  train: 1.000  test: 0.918

scale=    100.0  C=     10.0  nSV:   94  train: 0.965  test: 0.932
scale=    100.0  C=    100.0  nSV:   95  train: 0.987  test: 0.938
scale=    100.0  C=   1000.0  nSV:   92  train: 0.997  test: 0.929
scale=    100.0  C=  10000.0  nSV:   84  train: 0.965  test: 0.894

scale=    300.0  C=     10.0  nSV:   69  train: 0.911  test: 0.896
scale=    300.0  C=    100.0  nSV:   49  train: 0.946  test: 0.927
scale=    300.0  C=   1000.0  nSV:   48  train: 0.944  test: 0.919
scale=    300.0  C=  10000.0  nSV:   82  train: 0.891  test: 0.855

scale=   1000.0  C=     10.0  nSV:   62  train: 0.901  test: 0.885
scale=   1000.0  C=    100.0  nSV:   63  train: 0.898  test: 0.887
scale=   1000.0  C=   1000.0  nSV:  132  train: 0.884  test

In [ ]:
import numpy,sklearn,sklearn.datasets,numpy

D = sklearn.datasets.load_breast_cancer()
X = D['data']
T = D['target']
T = (D['target']==1)*2.0-1.0

for scale in [30,100,300,1000,3000]:
    for C in [10,100,1000,10000]:
        
        acctrain,acctest,nbsvs = [],[],[]
        
        svm = GaussianSVM(C=C,scale=scale)
        
        for i in range(10):

            # Split the data
            R = numpy.random.mtrand.RandomState(i).permutation(len(X))
            Xtrain,Xtest = X[R[:len(R)//2]]*1,X[R[len(R)//2:]]*1
            Ttrain,Ttest = T[R[:len(R)//2]]*1,T[R[len(R)//2:]]*1

            # Train and test the SVM
            svm.fit(Xtrain,Ttrain)
            acctrain += [(svm.predict(Xtrain)==Ttrain).mean()]
            acctest  += [(svm.predict(Xtest)==Ttest).mean()]
            nbsvs += [len(svm.X)*1.0]

        print('scale=%9.1f  C=%9.1f  nSV: %4d  train: %.3f  test: %.3f'%(
            scale,C,numpy.mean(nbsvs),numpy.mean(acctrain),numpy.mean(acctest)))
    print('')

scale=     30.0  C=     10.0  nSV:  183  train: 0.997  test: 0.921


scale=     30.0  C=    100.0  nSV:  178  train: 1.000  test: 0.918


scale=     30.0  C=   1000.0  nSV:  184  train: 1.000  test: 0.918


scale=     30.0  C=  10000.0  nSV:  182  train: 1.000  test: 0.918



scale=    100.0  C=     10.0  nSV:  117  train: 0.965  test: 0.935


...

scale=   1000.0  C=  10000.0  nSV:   38  train: 0.971  test: 0.951



scale=   3000.0  C=     10.0  nSV:   87  train: 0.912  test: 0.903


scale=   3000.0  C=    100.0  nSV:   68  train: 0.926  test: 0.919


scale=   3000.0  C=   1000.0  nSV:   58  train: 0.934  test: 0.929


scale=   3000.0  C=  10000.0  nSV:   49  train: 0.953  test: 0.943



We observe that the highest accuracy is obtained with a scale parameter that is neither too small nor too large. Best parameters are also often associated to a low number of support vectors.